### Slade Brooks
##### brooksl@mail.uc.edu
##### 02.03.2023
# AeroComputing Assignment 2



# Part 1

In [ ]:
# import shtuff
import matplotlib.pyplot as plt
import numpy as np
import time
import pandas
import sys

# import custom modules
sys.path.append("..//lib//")
import pyvot        # pivot direct methods
import pyterative       # iterative new cool epic methods

# Part 2

In [ ]:
# define a and b
a = np.zeros((20, 20), dtype="float64")
for i in range(20):
    a[i, i] = 4.
    if i < 19:
        a[i, i+1] = -1.
        a[i+1, i] = -1.
a[0, 19] = 1.
a[19, 0] = 1.

b = np.zeros((20, 1), dtype="float64")
b[19, 0] = 100.

In [ ]:
# compute each of direct methods

In [ ]:
# define interEqs fxn for gauss
def iterEqs(x, omega):
    n = len(x)
    x[0] = omega*(x[1] - x[n-1])/4.0 + (1.0 - omega)*x[0]
    for i in range(1, n-1):
        x[i] = omega*(x[i-1] + x[i+1])/4.0 + (1.0 - omega)*x[i]
    x[n-1] = omega*(100. - x[0] + x[n-2])/4.0 \
        + (1.0 - omega)*x[n-1]
    return x

# do gaussSeidel method
x = np.zeros(20)        # because n = 20
x,numIter,omega = pyterative.gaussSeidel(iterEqs,x)
print("\nNumber of iterations =",numIter)
print("\nRelaxation factor =",omega)
print("\nThe solution is:\n",x)

In [ ]:
# define Av for conj
def Ax(v):
    n = len(v)
    Ax = np.zeros(n)
    Ax[0] = 4.0*v[0] - v[1]+v[n-1]
    Ax[1:n-1] = -v[0:n-2] + 4.0*v[1:n-1] -v[2:n]
    Ax[n-1] = -v[n-2] + 4.0*v[n-1] + v[0]
    return Ax

# do conjGrad method
b = np.zeros(20)           # because n = 20
b[19] = 1.0
x = np.zeros(20)
x,numIter = pyterative.conjGrad(Ax,x,b)
print("\nThe solution is:\n",x)
print("\nNumber of iterations =",numIter)

# Part 3